In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
import json
import ast

In [2]:
# data_list = []

# # Path to the file
# file_path = 'goodreads_reviews_children.json.gz'

# with gzip.open(file_path, 'rt', encoding='utf-8') as file:
#     for line in file:
#         try:
#             # Safely evaluate each line as a dictionary
#             data_list.append(ast.literal_eval(line.strip()))
#         except Exception as e:
#             print(f"Error processing line: {line}\n{e}")

# # Display the first 5 entries to verify


In [3]:
data = []
file_path = 'renttherunway_final_data.json.gz'
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            # Safely evaluate each line as a dictionary
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(f"Error processing line: {line}\n{e}")



In [4]:
data[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [12]:
data = [
    entry for entry in data
    if all(field in entry and entry[field] for field in ['rating', 'review_text', 'review_summary', 'size', 'age', 'review_date'])
]

In [18]:
len(data)
data[:5]

[{'fit': 'fit',
  'user_id': '420272',
  'bust size': '34d',
  'item_id': '2260466',
  'weight': '137lbs',
  'rating': '10',
  'rented for': 'vacation',
  'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
  'body type': 'hourglass',
  'review_summary': 'So many compliments!',
  'category': 'romper',
  'height': '5\' 8"',
  'size': 14,
  'age': '28',
  'review_date': 'April 20, 2016'},
 {'fit': 'fit',
  'user_id': '273551',
  'bust size': '34b',
  'item_id': '153475',
  'weight': '132lbs',
  'rating': '10',
  'rented for': 'other',
  'review_text': 'I rented this dress for a photo shoot. The theme was "Hollywood Glam and Big Beautiful Hats". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event. ',
  'body type': 'straight & narr

In [24]:
def MSE(y, ypred):
    diffs = [(a-b)**2 for (a,b) in zip(y,ypred)]
    return sum(diffs) / len(diffs)

In [25]:
def feat1(d):
    return [1, len(d['review_text'])] 

In [26]:

dataTrain = data[:int(len(data)*0.8)]
dataTest = data[int(len(data)*0.8):]

In [27]:
Xtrain = [feat1(d) for d in dataTrain]
ytrain = [int(d['rating']) for d in dataTrain]
Xtest = [feat1(d) for d in dataTest]
ytest = [int(d['rating']) for d in dataTest]

In [28]:
max(ytest)
min(ytest)
unique_ratings = set(entry['rating'] for entry in data if 'rating' in entry)

# Display the unique rating values
print(unique_ratings)

{'6', '8', '2', '10', '4'}


In [30]:
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


2.0509410951628015

In [ ]:
def feat2(d):
    return [1, len(d['review_text']), len(d['review_summary'])] 

In [32]:
Xtrain = [feat2(d) for d in dataTrain]
Xtest = [feat2(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


2.043165781399794

In [34]:
def feat3(d):
    return [1, len(d['review_text']), len(d['review_summary']), d['review_text'].count('!') + d['review_summary'].count('!')] 

In [35]:
Xtrain = [feat3(d) for d in dataTrain]
Xtest = [feat3(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9290804311924041

In [36]:
def feat4(d):
    return [1, d['review_text'].count('!') + d['review_summary'].count('!')] 

In [37]:
Xtrain = [feat4(d) for d in dataTrain]
Xtest = [feat4(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.948420335125565